In [ ]:
import tkinter as tk
import random
import time


# Constantes y Variables Globales
TAMANO_GRID = 10
NUM_BOMBAS = 10

botones_grid = {} 
es_bomba = {} 
juego_terminado = False
inicio = 0
etiqueta_cronometro = None 

# Definiciones de Funciones

def contar_bombas_cercanas(r, c):
    """Cuenta el número de bombas en el radio 3x3 alrededor de (r, c)."""
    conteo = 0
    # Recorre en un radio de 3x3
    for filas in range(-1, 2):  # -1, 0, 1 para filas
        for columnas in range(-1, 2):  # -1, 0, 1 para columnas
            # Calcula las coordenadas de la casilla vecina
            fila_vecina, columna_vecina = r + filas, c + columnas

            # Asegúrate de que la casilla esté dentro de la grilla
            if 0 <= fila_vecina < TAMANO_GRID and 0 <= columna_vecina < TAMANO_GRID:
                if es_bomba[(r, c)]:
                    conteo += 1
    return conteo

def casilla_presionada(r, c):
    global juego_terminado
    
    if juego_terminado:
        return 

    boton_especifico = botones_grid[(r, c)]

    if es_bomba[(r, c)]:
        juego_terminado = True
        boton_especifico.config(text="💣", bg="red", state="disabled", relief="sunken")
        
        for (fila, col), boton in botones_grid.items():
            if es_bomba[(fila, col)]:
                boton.config(text="💣", bg="red")
    else:
        boton_especifico.config(state="disabled", relief="sunken", bg="lightgray")

def cronometro():
    global inicio, etiqueta_cronometro
    
    if juego_terminado:
        if etiqueta_cronometro:
             etiqueta_cronometro.config(text="Juego Terminado")
        return 
        
    tiempo_transcurrido = time.time() - inicio
    string_tiempo = time.strftime('%H:%M:%S', time.gmtime(tiempo_transcurrido))
    
    if etiqueta_cronometro:
        etiqueta_cronometro.config(text=f"Cronómetro {string_tiempo}")
    
    ventana.after(100, cronometro) 

def iniciar_juego():
    global inicio, juego_terminado, botones_grid, es_bomba, etiqueta_cronometro
    
    botones_grid = {}
    es_bomba = {}
    juego_terminado = False
    
    marco_etiquetas = tk.Frame(ventana)
    marco_etiquetas.pack(pady=10) 

    marco_juego = tk.Frame(ventana)
    marco_juego.pack(pady=10) 

    for i in range(TAMANO_GRID):
        for j in range(TAMANO_GRID):
            boton = tk.Button(marco_juego, text=" ", width=4, height=3)
            boton.grid(column=j, row=i)
            botones_grid[(i, j)] = boton
            es_bomba[(i, j)] = False
            boton.config(command=lambda r=i, c=j: casilla_presionada(r, c))

    coordenadas_posibles = list(botones_grid.keys())
    num_bombas_real = min(NUM_BOMBAS, len(coordenadas_posibles))
    coordenadas_bombas = random.sample(coordenadas_posibles, num_bombas_real)

    for (r, c) in coordenadas_bombas:
        es_bomba[(r, c)] = True
    
    etiqueta_cronometro = tk.Label(marco_etiquetas, text="Cronómetro", font=("Arial", 14))
    etiqueta_cronometro.pack()
    
    inicio = time.time()
    cronometro()

def seleccionar_dificultad_y_jugar(tamano, bombas):
    """Establece la dificultad, ajusta la ventana y comienza el juego"""
    global TAMANO_GRID, NUM_BOMBAS

    TAMANO_GRID = tamano
    NUM_BOMBAS = bombas

    if NUM_BOMBAS == 10:
        ventana.geometry("430x650+1100+0")
        ventana.minsize(430, 650)
    elif NUM_BOMBAS == 12:
        ventana.geometry("850x1250+680+0")
        ventana.minsize(800, 1000)
    elif NUM_BOMBAS == 20:
        ventana.geometry("1250x1800+400+0")
        ventana.minsize(1250, 1800)

    marco_opciones.pack_forget()
    iniciar_juego()

# Configuración de la Ventana Principal
ventana = tk.Tk()
ventana.title("Buscaminas")
ventana.geometry("700x800+480+0") 
ventana.minsize(300, 400)
ventana.iconbitmap("Buscaminas-meme.ico") 

# Pantalla de Opciones (Menú de Inicio)
marco_opciones = tk.Frame(ventana, pady=50)
marco_opciones.pack(expand=True)



etiqueta_titulo = tk.Label(marco_opciones, text="BUSCAMINAS", font=("Arial", 24, "bold"))
etiqueta_titulo.pack(pady=20)

etiqueta_subtitulo = tk.Label(marco_opciones, text="Selecciona una dificultad:", font=("Arial", 14))
etiqueta_subtitulo.pack(pady=10)

estilo_boton = {"font": ("Arial", 12), "width": 25, "pady": 10}

boton_facil = tk.Button(marco_opciones, text="Principiante (10x10, 10 bombas)", **estilo_boton,
                         command=lambda: seleccionar_dificultad_y_jugar(10, 10))
boton_facil.pack(pady=5)

boton_medio = tk.Button(marco_opciones, text="Intermedio (12x12, 12 bombas)", **estilo_boton,
                        command=lambda: seleccionar_dificultad_y_jugar(12, 12))
boton_medio.pack(pady=5)

boton_dificil = tk.Button(marco_opciones, text="Experto (15x15, 20 bombas)", **estilo_boton,
                          command=lambda: seleccionar_dificultad_y_jugar(15, 20))
boton_dificil.pack(pady=5)


ventana.mainloop()